In [12]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import json

app = JupyterDash(__name__)

df = pd.DataFrame({
    'ids': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'labels': ['A', 'B', 'B1', 'B2', 'C', 'C1'],
    'parents': ['', 'A', 'B', 'B', 'A', 'C'],
    'extra_info': ['info A', 'info B', 'info B1', 'info B2', 'info C', 'info C1'],
    'click_info': ['Information A', 'Information B', 
                   'Information B1', 'Information B2', 'Information C', 'Information C1']
})

fig = go.Figure(go.Treemap(
    ids = df['ids'],
    labels = df['labels'],
    parents = df['parents'],
    text = df['extra_info'].tolist()
))

app.layout = html.Div([
    dcc.Graph(
        id='my-graph',
        figure=fig
    ),
    html.Pre(id='click-data', style={'padding': '10px'}),
    html.Div(id='text-storage', style={'display': 'none'}),
    html.Div(id='node-clicked', style={'display': 'none'})
])

@app.callback(
    Output('my-graph', 'figure'),
    Output('text-storage', 'children'),
    Output('node-clicked', 'children'),
    Input('my-graph', 'clickData'),
    State('my-graph', 'figure'),
    State('text-storage', 'children'),
    State('node-clicked', 'children'))
def update_figure(clickData, fig, stored_text, clicked_node):
    if clickData:
        point_number = clickData['points'][0]['pointNumber']
        if stored_text is None:
            stored_text = json.dumps(fig['data'][0]['text'])
        if clicked_node is None or clicked_node != point_number:
            fig['data'][0]['text'] = json.loads(stored_text)
            fig['data'][0]['text'][point_number] = df.loc[point_number, 'click_info']
            clicked_node = point_number
        else:
            fig['data'][0]['text'] = json.loads(stored_text)
            clicked_node = None
    else:
        if stored_text is not None:
            fig['data'][0]['text'] = json.loads(stored_text)
        clicked_node = None

    return fig, stored_text, clicked_node

app.run_server(mode='inline')
# 功能合并版本

Dash is running on http://127.0.0.1:8050/



In [46]:
import plotly.graph_objects as go

# 创建一个简单的 Treemap 图表
labels = ['A', 'B', 'C']
values = [10, 20, 30]
custom_data = ['data1', 'data2', 'data3']

fig = go.Figure(go.Treemap(
    labels=labels,
    parents=['', '', ''],
    values=values,
    customdata=custom_data
))

# 设置悬停文本和自定义数据的显示格式
fig.update_traces(
    hovertemplate='Label: %{label}<br>Value: %{value}<br>Custom Data: %{customdata}'
)

# 显示图表
fig.show()


In [53]:
import plotly.graph_objects as go

# 创建一个简单的 Treemap 图表
labels = ['A', 'B', 'C']
values = [10, 20, 30]
custom_data = ['First line<br>Second line', 'Data with multiple lines', 'Single line']

fig = go.Figure(go.Treemap(
    labels=labels,
    parents=['', '', ''],
    values=values
))

# 添加自定义的文本注释
for i, text in enumerate(custom_data):
    annotation = go.layout.Annotation(
        x=0.5,  # x 坐标位置
        y=i+0.5,  # y 坐标位置
        text=text,  # 文本内容，包含换行
        showarrow=False,  # 不显示箭头
        font=dict(size=12)  # 字体大小
    )
    fig.add_annotation(annotation)

# 显示图表
fig.show()


In [57]:
import plotly.graph_objects as go

# 创建一个简单的 Treemap 图表
labels = ['A', 'B', 'C']
values = [10, 20, 30]
custom_data = ['First line<br>Second line', 'Data with multiple lines', 'Single line']

fig = go.Figure(go.Treemap(
    labels=labels,
    parents=['', '', ''],
    values=values,
    text=custom_data,  # 设置自定义数据为文本属性
    texttemplate='%{text}',  # 使用 texttemplate 属性来渲染自定义数据
     hoverinfo='none'  # 设置 hoverinfo 为 'none'，关闭悬停效果
))

# 显示图表
fig.show()


In [59]:
import plotly.graph_objects as go
from PIL import Image

# 创建一个简单的 Treemap 图表
labels = ['A', 'B', 'C']
values = [10, 20, 30]

fig = go.Figure(go.Treemap(
    labels=labels,
    parents=['', '', ''],
    values=values
))

# 加载自定义图片
image_path = "img/1.png.png"  # 替换为您自己的图片路径
image = Image.open(image_path)

# 获取 Treemap 图表的布局信息
layout = fig.layout

# 添加图片对象到每个矩形的注释中
for i, label in enumerate(labels):
    # 计算注释的坐标位置
    x = layout.treemap[i].x
    y = layout.treemap[i].y
    width = layout.treemap[i].width
    height = layout.treemap[i].height

    # 创建图片对象
    annotation = go.layout.Annotation(
        x=x + width/2,
        y=y + height/2,
        xref='x',
        yref='y',
        text='',
        showarrow=False,
        font=dict(color='rgba(0, 0, 0, 0)'),  # 设置字体颜色为透明，使图片覆盖整个矩形
        align='center',
        opacity=1,
        xanchor='center',
        yanchor='middle',
        xclick=x + width/2,
        yclick=y + height/2,
        captureevents=True,
        hovertext=label,  # 设置悬停时显示的标签
        hoverlabel=dict(
            bgcolor='white',  # 设置悬停时显示的标签背景颜色为白色
